In [ ]:
%load_ext autoreload
%autoreload 2
%env CUDA_LAUNCH_BLOCKING=1
import torch

from train import init_model, init_training, create_connections_datasets

In [63]:
tokenizer, model = init_model()

2025-05-06 21:15:47.835 | INFO     | train:init_model:4 - Model loaded.
2025-05-06 21:15:47.842 | INFO     | train:init_model:7 - Using device cuda:0


In [66]:
train_dataset, _ = create_connections_datasets(tokenizer)

In [4]:
training_objects = init_training(model, train_dataset)

In [67]:
from train import init_training
from grpo import rollout, update_policy

from tasks.countdown import reward_function

to = init_training(model, train_dataset)
batch = next(iter(to.train_dataloader))

episodes = rollout(
    model=model,
    tokenizer=tokenizer,
    batch=batch,
    max_new_tokens=to.config.max_new_tokens,
    num_answer_per_question=to.config.num_answer_per_question,
    reward_function=reward_function,
    device=to.device,
)
if to.config.skip_unfinished_episodes:
    episodes = [episode for episode in episodes if episode.is_finished]

2025-05-06 21:16:38.984 | INFO     | grpo:rollout:43 - Generating responses for 2 prompts, max_tokens=256
2025-05-06 21:17:35.359 | INFO     | grpo:rollout:74 - Generated token ids: 256
2025-05-06 21:17:35.361 | INFO     | grpo:rollout:81 - Generated token ids after removing padding: 256
2025-05-06 21:17:35.362 | INFO     | grpo:rollout:88 - Generated text: 

Okay, let's see. The user provided a list of words and wants me to group them into four groups of four, each related. Each word can only be in one group. Let me start by looking at the words.

First, the words are: caw, chirp, cluck, tweet, bunk, crock, hogwash, horsefeathers, fruit, pitcher, skull, tablecloth, contact, message, ping, text.

Hmm, some of these sounds like animals. Caw, chirp, cluck, tweet, cluck... Wait, cluck is a common animal sound. Also, cricket, cricket... but the words here are "horsefeathers". Wait, "horsefeathers" is a term for horse feathers, but maybe not directly related. Let me think.

Another angle: "

rewards {'reward': 0.0, 'reward_info': {'format_reward': 0.0, 'answer_reward': 0.0}}
rewards {'reward': 0.0, 'reward_info': {'format_reward': 0.0, 'answer_reward': 0.0}}


In [ ]:
episodes[0].reward

In [ ]:
# Update policy - compute loss and perform backward pass
results = update_policy(
    model=model,
    optimizer=to.optimizer,
    episodes=episodes,
    micro_batch_size=to.config.micro_batch_size,
    pad_token_id=tokenizer.pad_token_id,
    max_grad_norm=to.config.max_grad_norm,
    device=to.device,
    dtype=to.dtype,
    apply_loss=True
)


In [ ]:
results